# Make SARS-CoV-2 N protein - Cytokine Complex Tasks

Note: This notebook is to be run from the `./inputs` folder using a Python kernel with PyMOL installed.

In [3]:
!conda install -y -c conda-forge -c schrodinger pymol-bundle pandas

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/colby/SARS-CoV-2_N-Cytokine_Docking/.conda

  added / updated specs:
    - pandas
    - pymol-bundle


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    apbs-1.5                   |       h1de35cc_3         258 KB  schrodinger
    biopython-1.81             |   py39ha30fb19_0         2.6 MB  conda-forge
    brotli-1.0.9               |       hb7f2c08_8          19 KB  conda-forge
    brotli-bin-1.0.9           |       hb7f2c08_8          17 KB  conda-forge
    c-ares-1.19.1              |       h0dc2134_0         101 KB  conda-forge
    certifi-2023.5.7           |     pyhd8ed1ab_0         149 KB  conda-forge
    charset-normalizer-3

In [1]:
import os, shutil, sys
import pandas as pd
from itertools import product
from pymol import cmd
from random import sample

In [2]:
## List N protein files
n_files = os.listdir('../inputs/N-Proteins/')
n_files = [x.replace('.pdb', '') for x in n_files]

## TEMP SUBSET FOR TESTING
# n_files = ['SARS-CoV-2-WA1-N', 'SARS-CoV-2-XBB-N']

## List Cytokine files (excluding multichain ones)
multichain_cytokines = [
    'IL-12p70.pdb',
    'IL-12p70_renumbered.pdb',
    'IL-23.pdb',
    'IL-23_renumbered.pdb',
    'IL-27.pdb',
    'IL-27_renumbered.pdb',
    'IL-35.pdb',
    'IL-35_renumbered.pdb'
]

cytokine_files = os.listdir('../inputs/Cytokines/')
cytokine_files = [x.replace('.pdb', '') for x in cytokine_files if x not in multichain_cytokines]

## TEMP SUBSET FOR TESTING
# cytokine_files = ['CXCL12alpha', 'CXCL12beta']

files_dict = {
    'n_protein': n_files,
    'cytokine_protein': cytokine_files
}

print(files_dict)

{'n_protein': ['BANAL-20-52-N', 'MERS-CoV-N', 'OC43-N', 'RaTG13-N', 'SARS-CoV-2-B.1.1-N', 'SARS-CoV-2-B.1.1.529-N', 'SARS-CoV-2-B.1.1.7-N', 'SARS-CoV-2-B.1.351-N', 'SARS-CoV-2-B.1.617.2-DeltaA-N', 'SARS-CoV-2-BA.1.1-N', 'SARS-CoV-2-BA.2-N', 'SARS-CoV-2-BA.4-N', 'SARS-CoV-2-BQ.1-N', 'SARS-CoV-2-P.1-N', 'SARS-CoV-2-WA1-N', 'SARS-CoV-2-XBB-N', 'SARS-CoV-N'], 'cytokine_protein': ['CCL1', 'CCL11', 'CCL13', 'CCL14', 'CCL15', 'CCL16', 'CCL17', 'CCL18', 'CCL19', 'CCL2', 'CCL20', 'CCL21', 'CCL22', 'CCL23', 'CCL24', 'CCL25', 'CCL26', 'CCL27', 'CCL28', 'CCL3', 'CCL3L1', 'CCL4', 'CCL4L1', 'CCL5', 'CCL7', 'CCL8', 'CX3CL1', 'CXCL1', 'CXCL10', 'CXCL11', 'CXCL12alpha', 'CXCL12beta', 'CXCL13', 'CXCL14', 'CXCL16', 'CXCL2', 'CXCL3', 'CXCL4', 'CXCL5', 'CXCL6', 'CXCL7', 'CXCL8', 'CXCL9', 'IFNA2', 'IFNbeta', 'IFNgamma', 'IFNlambda1', 'IFNomega', 'IL-10', 'IL-13', 'IL-17a', 'IL-18', 'IL-18BP', 'IL-1alpha', 'IL-1beta', 'IL-6', 'IL-6Ralpha', 'IL6', 'INFA2', 'INFbeta', 'TNFalpha', 'TNFbeta', 'XCL1']}


In [3]:
## Prepare Task DataFrame
task_grid = pd.DataFrame([row for row in product(*files_dict.values())], columns=files_dict.keys())

task_grid['complex_id'] = task_grid['n_protein'].replace('SARS-CoV-2-N-', '') + "__" + task_grid['cytokine_protein']
task_grid['n_pdb'] = task_grid['n_protein'] + ".pdb"
task_grid['cytokine_pdb'] = task_grid['cytokine_protein'] + ".pdb"

## Make Empty Columns
task_grid['experiment_path'] = ''
task_grid['n_residues'] = ''
task_grid['cytokine_residues'] = ''

## Reorder Columns
task_grid = task_grid[[
    'complex_id',
    'experiment_path',
    'n_protein', 'n_pdb', 'n_residues',
    'cytokine_protein', 'cytokine_pdb', 'cytokine_residues'
    ]]

display(task_grid)

,complex_id,experiment_path,n_protein,n_pdb,n_residues,cytokine_protein,cytokine_pdb,cytokine_residues
0,BANAL-20-52-N__CCL1,,BANAL-20-52-N,BANAL-20-52-N.pdb,,CCL1,CCL1.pdb,
1,BANAL-20-52-N__CCL11,,BANAL-20-52-N,BANAL-20-52-N.pdb,,CCL11,CCL11.pdb,
2,BANAL-20-52-N__CCL13,,BANAL-20-52-N,BANAL-20-52-N.pdb,,CCL13,CCL13.pdb,
3,BANAL-20-52-N__CCL14,,BANAL-20-52-N,BANAL-20-52-N.pdb,,CCL14,CCL14.pdb,
4,BANAL-20-52-N__CCL15,,BANAL-20-52-N,BANAL-20-52-N.pdb,,CCL15,CCL15.pdb,
...,...,...,...,...,...,...,...,...
1066,SARS-CoV-N__INFA2,,SARS-CoV-N,SARS-CoV-N.pdb,,INFA2,INFA2.pdb,
1067,SARS-CoV-N__INFbeta,,SARS-CoV-N,SARS-CoV-N.pdb,,INFbeta,INFbeta.pdb,
1068,SARS-CoV-N__TNFalpha,,SARS-CoV-N,SARS-CoV-N.pdb,,TNFalpha,TNFalpha.pdb,
1069,SARS-CoV-N__TNFbeta,,SARS-CoV-N,SARS-CoV-N.pdb,,TNFbeta,TNFbeta.pdb,


In [4]:
## Define Helper Functions

## Find Random Surface Residues
def find_random_surface_residues(file, percentage = 0.25):
    cmd.load(file)
    ## Finds atoms on the surface of a protein
    ## Logic from: https://pymolwiki.org/index.php/FindSurfaceResidues
    cutoff = 2.0
    tmpObj = cmd.get_unused_name("_tmp")
    cmd.create(tmpObj, "(all) and polymer", zoom=0)
    cmd.set("dot_solvent", 1, tmpObj)
    cmd.get_area(selection=tmpObj, load_b=1)
    cmd.remove(tmpObj + " and b < " + str(cutoff))
    cmd.select("exposed_atoms", "(all) in " + tmpObj)
    cmd.delete(tmpObj)
    ## Get a list of residue numbers and then subset it
    surface_residues = set()
    cmd.iterate('exposed_atoms', "surface_residues.add(resi)", space=locals())
    k = int(len(surface_residues) * percentage)
    surface_residues_sample = list(sample(surface_residues, k))
    surface_residues_sample = [int(x) for x in surface_residues_sample]
    surface_residues_sample.sort()
    ## Reintialize Everything
    cmd.reinitialize(what='everything')
    return surface_residues_sample

## Write AIR File
def write_air_file(active1, passive1, active2, passive2, segid1='A', segid2='B', output_file = "air.tbl"):

    active1 = [int(x) for x in active1]
    passive1 = [int(x) for x in passive1]
    active2 = [int(x) for x in active2]
    passive2 = [int(x) for x in passive2]

    all1 = active1 + passive1
    all2 = active2 + passive2

    param_lines = []

    for resi1 in active1:
        param_lines.append('assign (resi {:d} and segid {:s})'.format(resi1, segid1))
        param_lines.append('(')
        c = 0
        for resi2 in all2:
            param_lines.append('       (resi {:d} and segid {:s})'.format(resi2, segid2))
            c += 1
            if c != len(all2):
                param_lines.append('        or')
        param_lines.append(') 2.0 2.0 0.0\n')
            
    for resi2 in active2:
        param_lines.append('assign (resi {:d} and segid {:s})'.format(resi2, segid2))
        param_lines.append('(\n')
        c = 0
        for resi1 in all1:
            param_lines.append('       (resi {:d} and segid {:s})'.format(resi1, segid1))
            c += 1
            if c != len(all1):
                param_lines.append('        or\n')
        param_lines.append(') 2.0 2.0 0.0\n')
    
    f = open(output_file, "w")
    f.writelines("\n".join(param_lines))
    f.close()

## Write run.param file
def write_run_params(ambig_tbl = None,
                     haddock_dir = "/root/haddock/haddock2.4-2021-01/",
                     n_comp = 2,
                     pdb_file_1 = "",
                     pdb_file_2 = "",
                     project_dir = "./",
                     prot_segid_1 = "A",
                     prot_segid_2 = "B",
                     run_number = 1,
                     output_file = "run.param"):
    param_lines  = [
        f"HADDOCK_DIR={haddock_dir}",
        f"N_COMP={n_comp}",
        f"PDB_FILE1={pdb_file_1}",
        f"PDB_FILE2={pdb_file_2}",
        f"PROJECT_DIR={project_dir}",
        f"PROT_SEGID_1={prot_segid_1}",
        f"PROT_SEGID_2={prot_segid_2}",
        f"RUN_NUMBER={run_number}",
    ]
    if ambig_tbl:
        param_lines = [f"AMBIG_TBL={ambig_tbl}", *param_lines]
    # print(param_lines)
    # return(param_lines)
    f = open(output_file, "w")
    f.writelines("\n".join(param_lines))
    f.close()
    return(True)


In [5]:
## Setup Experiment Paths
# root_path = '../cluster_tests/'
# root_path = '../experiments/'
root_path = '../100pct_full_experiments/'

for index, complex in task_grid.iterrows():
    print(f"Prepping: {complex['complex_id']}...\n")
    experiment_path = root_path + complex['complex_id']
    task_grid.at[index,'experiment_path'] = experiment_path

    ## Make Experiment Folder
    os.makedirs(experiment_path, exist_ok = True)

    ## Copy PDBs
    print("Copying PDB files...")
    shutil.copyfile("../inputs/N-Proteins/" + complex['n_pdb'],
                    experiment_path + "/" + complex['n_pdb'])
    shutil.copyfile("../inputs/Cytokines/" + complex['cytokine_pdb'],
                    experiment_path + "/" + complex['cytokine_pdb'])
    
    ## Copy ana_scripts folder
    print("Copying ana_scripts folder...")
    shutil.copytree("../helper_scripts/ana_scripts/",
                    experiment_path + "/ana_scripts/")
    
    ## Make run.cns.path file
    shutil.copyfile("../helper_scripts/run.cns.patch",
                    experiment_path + "/run.cns.patch")
    
    ## Make run-docking.csh file
    shutil.copyfile("../helper_scripts/run-docking.csh",
                    experiment_path + "/run-docking.csh")
    
    ## Define Random Restraints
    print("Finding random N surface residues...")
    N_residues = find_random_surface_residues(file = f"../inputs/N-Proteins/{complex['n_pdb']}", percentage = 1.00)
    task_grid.at[index,'n_residues'] = '`' + str(','.join(str(x) for x in N_residues))
    print("Finding random Cytokine surface residues...")
    cytokine_residues = find_random_surface_residues(file = f"../inputs/Cytokines/{complex['cytokine_pdb']}", percentage = 1.00)
    task_grid.at[index,'cytokine_residues'] = f"`{','.join(str(x) for x in cytokine_residues)}"

    ## Write out AIR file
    print("Writing AIR file...")
    write_air_file(active1 = N_residues,
                   passive1 = [],
                   active2 = cytokine_residues,
                   passive2 = [],
                   segid1='A', segid2='B',
                   output_file = f"{experiment_path}/air.tbl")

    ## Make run.param file
    print("Writing run.param file...")
    write_run_params(ambig_tbl = "./air.tbl",
                    haddock_dir = "/root/haddock/haddock2.4-2021-01/",
                    n_comp = 2,
                    pdb_file_1 = f"./{complex['n_pdb']}",
                    pdb_file_2 = f"./{complex['cytokine_pdb']}",
                    project_dir = "./",
                    prot_segid_1 = "A",
                    prot_segid_2 = "B",
                    run_number = 1,
                    output_file = f"{experiment_path}/run.param")
    
    ## Copy cleanup Python script
    print("Copying cleanup.py script...")
    shutil.copyfile("../helper_scripts/cleanup.py",
                    experiment_path + "/cleanup.py")
    
    print(f"Finished Prepping: {complex['complex_id']}.\n")

Prepping: BANAL-20-52-N__CCL1...

Copying PDB files...
Copying ana_scripts folder...
Finding random N surface residues...
 PyMOL not running, entering library mode (experimental)
Finding random Cytokine surface residues...
Writing AIR file...
Writing run.param file...
Copying cleanup.py script...
Finished Prepping: BANAL-20-52-N__CCL1.

Prepping: BANAL-20-52-N__CCL11...

Copying PDB files...
Copying ana_scripts folder...
Finding random N surface residues...
Finding random Cytokine surface residues...
Writing AIR file...
Writing run.param file...
Copying cleanup.py script...
Finished Prepping: BANAL-20-52-N__CCL11.

Prepping: BANAL-20-52-N__CCL13...

Copying PDB files...
Copying ana_scripts folder...
Finding random N surface residues...
Finding random Cytokine surface residues...
Writing AIR file...
Writing run.param file...
Copying cleanup.py script...
Finished Prepping: BANAL-20-52-N__CCL13.

Prepping: BANAL-20-52-N__CCL14...

Copying PDB files...
Copying ana_scripts folder...
Finding

In [12]:
## Create Analysis CSV
# task_grid.to_csv(f'{root_path}/100pct_full_analyses.csv', index=False)